# Explore Sleeper API, pull fantasy and draft history data

This notebook will go through the PWL data and look for historical trends. This will give experience in parsing JSON files and utilizing functions to access an API.

https://docs.sleeper.app/

https://docs.python.org/3/library/json.html

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import read_json
pd.set_option('display.max_columns', None)

### Pull Player Data

First, we will get data for the players. This is done at a maximum of once per day via request of Sleeper. We save this into a `.json` file so we do not need to constantly call it.

In [2]:
"""
players = 'https://api.sleeper.app/v1/players/nfl'
players = requests.get(players)
players = json.loads(players.text)
json.dump(players, open('nfl_players.json', 'w'), indent=2)
"""
print('Call player info once a day, max. Be a courteous API user!')

Call player info once a day, max. Be a courteous API user!


In [3]:
# load player json to pd dataframe, current as of 2021-08-31
players = read_json('nfl_players.json', orient='index')

# clean up relevant columns
players_cols = ['player_id', 'age', 'full_name', 'team', 'college', 'fantasy_positions', 'height', 'weight', 'years_exp']
players = players[players_cols].reset_index(drop=True)

In [4]:
players

,player_id,age,full_name,team,college,fantasy_positions,height,weight,years_exp
0,2103,27.0,Cody Booth,None,Temple,[OL],"6'5""",285,1.0
1,6250,NaN,Eurndraus Bryant,None,None,[DL],,,0.0
2,4116,26.0,Antonio Garcia,None,Troy,[OL],"6'6""",293,3.0
3,6177,25.0,Khalen Saunders,KC,Western Illinois,[DL],"6'0""",324,2.0
4,5870,24.0,Daniel Jones,NYG,Duke,[QB],"6'5""",220,2.0
...,...,...,...,...,...,...,...,...,...
8085,CHI,NaN,NaN,CHI,NaN,[DEF],NaN,NaN,NaN
8086,KC,NaN,NaN,KC,NaN,[DEF],NaN,NaN,NaN
8087,BUF,NaN,NaN,BUF,NaN,[DEF],NaN,NaN,NaN
8088,MIN,NaN,NaN,MIN,NaN,[DEF],NaN,NaN,NaN


### Pull Rosters and Users

Now, we get info pertaining to our league specifically. We pull our current rosters (2021) and the current users in the league. We append the rosters and users to our base url.

In [5]:
league_id = '732830961967939584'
base_url = 'https://api.sleeper.app/v1/league/' + league_id
rosters = base_url + '/rosters/'
users = base_url + '/users/'

# obtain html requests from rosters and users
rosters = requests.get(rosters)
users = requests.get(users)

# pull the JSON of the html request
rosters = json.loads(rosters.text)
users = json.loads(users.text)

# convert the jsons to pandas tables
rosters = pd.json_normalize(rosters)
users = pd.json_normalize(users)

# expand list of players to create 1 row per player by user
# basically an opposite of "groupby"
rosters = rosters.explode('players').reset_index(drop=True)

# one-hot variable if player is starter or not
rosters['is_starter'] = [1 if pid in start_id else
                         0 for pid, start_id in zip(rosters['players'], rosters['starters'])]
# only keep relevant columns for rosters
rosters_cols = ['owner_id', 'players', 'is_starter']
rosters = rosters[rosters_cols]

# only keep relevant columns for users
users_cols = ['user_id', 'display_name', 'metadata.team_name']
users = users[users_cols]

Let's take a look at the `rosters` and `users` table after sorting out these JSONs.

In [6]:
rosters

,owner_id,players,is_starter
0,328344236795654144,2197,0
1,328344236795654144,2505,1
2,328344236795654144,3664,0
3,328344236795654144,4199,1
4,328344236795654144,4984,1
...,...,...,...
179,606305973077884928,6770,0
180,606305973077884928,6789,0
181,606305973077884928,7526,1
182,606305973077884928,7564,1


In [7]:
users

,user_id,display_name,metadata.team_name
0,66280610104492032,mikesully15,Antibody Boyz FC
1,200116759896788992,Tshah,Fields of Dreams
2,328196718921265152,mattmarch27,Defending Champ
3,328344236795654144,kbitner3,Bitner Beef
4,464880277399793664,Dannyg007,The Gunslingers
5,466304211961769984,jacobgeiger21,Jameis1of1
6,515050118498893824,maxmorrison,The Mixon Administration
7,591495481419419648,jcbalo,#NagyOut
8,603415394434228224,PEW,NaN
9,603764838145011712,stendervon,🐗


We have some incredibly helpful and relevant information for our league. We can get more information about the players and users from the other two JSONs we pulled earlier. We will first include user info, then player info. This will be saved into the `pwl` object for the name of our league.

In [8]:
pwl = pd.merge(rosters, users, how='left', left_on='owner_id', right_on='user_id')
pwl = pd.merge(pwl, players, how='left', left_on='players', right_on='player_id')
pwl = pwl.drop(['owner_id', 'players'], axis=1)
pwl = pwl.explode('fantasy_positions')
pwl

,is_starter,user_id,display_name,metadata.team_name,player_id,age,full_name,team,college,fantasy_positions,height,weight,years_exp
0,0,328344236795654144,kbitner3,Bitner Beef,2197,27.0,Brandin Cooks,HOU,Oregon State,WR,"5'10""",183,7.0
1,1,328344236795654144,kbitner3,Bitner Beef,2505,28.0,Darren Waller,LV,Georgia Tech,TE,"6'6""",255,6.0
2,0,328344236795654144,kbitner3,Bitner Beef,3664,28.0,J.D. McKissic,WAS,Arkansas State,RB,"5'10""",195,5.0
3,1,328344236795654144,kbitner3,Bitner Beef,4199,26.0,Aaron Jones,GB,Texas-El Paso,RB,"5'9""",208,4.0
4,1,328344236795654144,kbitner3,Bitner Beef,4984,25.0,Josh Allen,BUF,Wyoming,QB,"6'5""",237,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0,606305973077884928,dmarchio,JK About Contending,6770,24.0,Joe Burrow,CIN,Louisiana State,QB,"6'4""",216,1.0
180,0,606305973077884928,dmarchio,JK About Contending,6789,22.0,Henry Ruggs,LV,Alabama,WR,"6'0""",190,1.0
181,1,606305973077884928,dmarchio,JK About Contending,7526,22.0,Jaylen Waddle,MIA,Alabama,WR,"5'10""",182,0.0
182,1,606305973077884928,dmarchio,JK About Contending,7564,21.0,Ja'Marr Chase,CIN,LSU,WR,"6'0""",208,0.0


### Get Draft Data

This will be helpful in attaching the draft positions of the players for each team. We can then pull data on ADP and where a player "should" be drafted given our league's settings. We can also pull league settings via Sleeper API as well.

In [13]:
draft_2021_id = '732830961967939585'
draft_2020_id = '601085963854196737'

def get_draft(draft_id):
    draft = 'https://api.sleeper.app/v1/draft/{}/picks'.format(draft_id)
    draft = requests.get(draft)
    draft = json.loads(draft.text)
    draft = pd.json_normalize(draft)
    draft_cols = ['round', 'player_id', 'picked_by', 'pick_no']
    draft = draft[draft_cols]
    return draft

draft_2021 = get_draft(draft_2021_id)
draft_2020 = get_draft(draft_2020_id)

draft_2021 = pd.merge(draft_2021, pwl, how='right', left_on='player_id', right_on='player_id')
draft_2021 = draft_2021.drop('player_id', axis=1)

In [14]:
draft_2021

,round,picked_by,pick_no,is_starter,user_id,display_name,metadata.team_name,age,full_name,team,college,fantasy_positions,height,weight,years_exp
0,9.0,328344236795654144,104.0,0,328344236795654144,kbitner3,Bitner Beef,27.0,Brandin Cooks,HOU,Oregon State,WR,"5'10""",183,7.0
1,5.0,328344236795654144,56.0,1,328344236795654144,kbitner3,Bitner Beef,28.0,Darren Waller,LV,Georgia Tech,TE,"6'6""",255,6.0
2,13.0,328344236795654144,152.0,0,328344236795654144,kbitner3,Bitner Beef,28.0,J.D. McKissic,WAS,Arkansas State,RB,"5'10""",195,5.0
3,1.0,328344236795654144,8.0,1,328344236795654144,kbitner3,Bitner Beef,26.0,Aaron Jones,GB,Texas-El Paso,RB,"5'9""",208,4.0
4,3.0,328344236795654144,32.0,1,328344236795654144,kbitner3,Bitner Beef,25.0,Josh Allen,BUF,Wyoming,QB,"6'5""",237,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,13.0,606305973077884928,148.0,0,606305973077884928,dmarchio,JK About Contending,24.0,Joe Burrow,CIN,Louisiana State,QB,"6'4""",216,1.0
180,11.0,606305973077884928,124.0,0,606305973077884928,dmarchio,JK About Contending,22.0,Henry Ruggs,LV,Alabama,WR,"6'0""",190,1.0
181,8.0,606305973077884928,93.0,1,606305973077884928,dmarchio,JK About Contending,22.0,Jaylen Waddle,MIA,Alabama,WR,"5'10""",182,0.0
182,5.0,606305973077884928,52.0,1,606305973077884928,dmarchio,JK About Contending,21.0,Ja'Marr Chase,CIN,LSU,WR,"6'0""",208,0.0


In [19]:
draft_2021[draft_2021.full_name=='Ty Johnson']

,round,picked_by,pick_no,is_starter,user_id,display_name,metadata.team_name,age,full_name,team,college,fantasy_positions,height,weight,years_exp
55,NaN,NaN,NaN,0,200116759896788992,Tshah,Fields of Dreams,23.0,Ty Johnson,NYJ,Maryland,RB,"5'10""",210,2.0
